* Dataset Source: https://github.com/TheEconomist/big-mac-data
* Analysis Source: https://www.kaggle.com/iampedroalz/the-big-mac-index

In [1]:
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)

1.1.0
1.18.5


In [2]:
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.graph_objects as go
from IPython.display import Image
from sklearn.linear_model import LinearRegression
from scipy import stats

#### Codebook

This codebook largely applies to all three files. The exception is the variables suffixed "\_raw" or "\_adjusted"—these appear (with suffixes) in the "full" file but without suffixes in the respective ("raw" or "adjusted") files.

| variable      | definition                                            | source                     |
| ------------- | ----------------------------------------------------- | -------------------------- |
| date          | Date of observation                                   |
| iso_a3        | Three-character [ISO 3166-1 country code][iso 3166-1] |
| currency_code | Three-character [ISO 4217 currency code][iso 4217]    |
| name          | Country name                                          |
| local_price   | Price of a Big Mac in the local currency              | McDonalds; _The Economist_ |
| dollar_ex     | Local currency units per dollar                       | _Reuters_                  |
| dollar_price  | Price of a Big Mac in dollars                         |
| USD_raw       | Raw index, relative to the US dollar                  |
| EUR_raw       | Raw index, relative to the Euro                       |
| GBP_raw       | Raw index, relative to the British pound              |
| JPY_raw       | Raw index, relative to the Japanese yen               |
| CNY_raw       | Raw index, relative to the Chinese yuan               |
| GDP_dollar    | GDP per person, in dollars                            | IMF                        |
| adj_price     | GDP-adjusted price of a Big Mac, in dollars           |
| USD_adjusted  | Adjusted index, relative to the US dollar             |
| EUR_adjusted  | Adjusted index, relative to the Euro                  |
| GBP_adjusted  | Adjusted index, relative to the British pound         |
| JPY_adjusted  | Adjusted index, relative to the Japanese yen          |
| CNY_adjusted  | Adjusted index, relative to the Chinese yuan          |


In [3]:
big_mac_countries = ['ARG', 'AUS', 'BRA', 'GBR', 'CAN', 'CHL', 'CHN', 'CZE', 'DNK',
                      'EGY', 'HKG', 'HUN', 'IDN', 'ISR', 'JPN', 'MYS', 'MEX', 'NZL',
                      'NOR', 'PER', 'PHL', 'POL', 'RUS', 'SAU', 'SGP', 'ZAF', 'KOR',
                      'SWE', 'CHE', 'TWN', 'THA', 'TUR', 'ARE', 'USA', 'COL', 'CRI',
                      'PAK', 'LKA', 'UKR', 'URY', 'IND', 'VNM', 'GTM', 'HND',
                      'NIC', 'AZE', 'BHR', 'HRV', 'JOR', 'KWT', 'LBN', 'MDA', 'OMN',
                      'QAT', 'ROU', 'EUZ']


In [17]:
big_mac_full = pd.read_csv('./data/big-mac-full-index.csv')
big_mac_full.tail(3)

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
1383,2020-07-01,USA,USD,United States,5.71,1.0000,5.710000,0.00000,0.19303,0.33494,0.57062,0.84286,62868.917,5.174502,0.000,-0.019,0.122,0.279,0.070
1384,2020-07-01,VNM,VND,Vietnam,66000.00,23180.0000,2.847282,-0.50135,-0.40510,-0.33433,-0.21681,-0.08106,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1385,2020-07-01,ZAF,ZAR,South Africa,31.00,16.6725,1.859349,-0.67437,-0.61151,-0.56530,-0.48856,-0.39991,6353.846,2.871878,-0.413,-0.425,-0.342,-0.249,-0.372


In [18]:
big_mac = pd.read_csv('./data/big-mac-source-data.csv')
big_mac.tail(3)

,name,iso_a3,currency_code,local_price,dollar_ex,GDP_dollar,date
1655,United States,USA,USD,5.71,1.0000,62868.917,2020-07-01
1656,Vietnam,VNM,VND,66000.00,23180.0000,2551.123,2020-07-01
1657,South Africa,ZAF,ZAR,31.00,16.6725,6353.846,2020-07-01


In [21]:
print(list(big_mac.columns))

['name', 'iso_a3', 'currency_code', 'local_price', 'dollar_ex', 'GDP_dollar', 'date', 'dollar_price']


In [22]:
big_mac["dollar_price"] = big_mac["local_price"] / big_mac["dollar_ex"]
big_mac = big_mac[['name', 'iso_a3', 'currency_code', 'local_price', 'dollar_ex', 'dollar_price', 'GDP_dollar', 'date']]
big_mac.tail()

,name,iso_a3,currency_code,local_price,dollar_ex,dollar_price,GDP_dollar,date
1653,Ukraine,UKR,UAH,59.00,27.1300,2.174714,3112.883,2020-07-01
1654,Uruguay,URY,UYU,189.00,43.6750,4.327418,17014.132,2020-07-01
1655,United States,USA,USD,5.71,1.0000,5.710000,62868.917,2020-07-01
1656,Vietnam,VNM,VND,66000.00,23180.0000,2.847282,2551.123,2020-07-01
1657,South Africa,ZAF,ZAR,31.00,16.6725,1.859349,6353.846,2020-07-01


In [24]:
big_mac_2020 = big_mac[big_mac['date']==big_mac['date'].max()]
big_mac_2020.tail(3)

,name,iso_a3,currency_code,local_price,dollar_ex,dollar_price,GDP_dollar,date
1655,United States,USA,USD,5.71,1.0000,5.710000,62868.917,2020-07-01
1656,Vietnam,VNM,VND,66000.00,23180.0000,2.847282,2551.123,2020-07-01
1657,South Africa,ZAF,ZAR,31.00,16.6725,1.859349,6353.846,2020-07-01


In [25]:
# look up dollar nominal bic mac price in base currency
base_currency = "KRW"
currency_price = float(big_mac_2020.loc[big_mac_2020["currency_code"]== base_currency, "dollar_price"])
currency_price

3.7470335984012655

In [26]:
# comparing dollar price of Korea to other countries
big_mac_2020["currency_price"] = currency_price
big_mac_2020["percentage_gap"] = big_mac_2020["dollar_price"] / currency_price - 1

In [27]:
big_mac_2020 = big_mac_2020.round(decimals = 3)
big_mac_2020.tail(3)

,name,iso_a3,currency_code,local_price,dollar_ex,dollar_price,GDP_dollar,date,currency_price,percentage_gap
1655,United States,USA,USD,5.71,1.000,5.710,62868.917,2020-07-01,3.747,0.524
1656,Vietnam,VNM,VND,66000.00,23180.000,2.847,2551.123,2020-07-01,3.747,-0.240
1657,South Africa,ZAF,ZAR,31.00,16.672,1.859,6353.846,2020-07-01,3.747,-0.504


In [28]:
big_mac_2020[["name", "iso_a3", "local_price", "dollar_ex", "dollar_price", "currency_price",  "percentage_gap"]]

,name,iso_a3,local_price,dollar_ex,dollar_price,currency_price,percentage_gap
1586,United Arab Emirates,ARE,14.75,3.673,4.016,3.747,0.072
1587,Argentina,ARG,250.00,71.241,3.509,3.747,-0.063
1588,Australia,AUS,6.55,1.431,4.578,3.747,0.222
1589,Austria,AUT,3.86,0.880,4.388,3.747,0.171
1590,Azerbaijan,AZE,3.95,1.699,2.325,3.747,-0.380
...,...,...,...,...,...,...,...
1653,Ukraine,UKR,59.00,27.130,2.175,3.747,-0.420
1654,Uruguay,URY,189.00,43.675,4.327,3.747,0.155
1655,United States,USA,5.71,1.000,5.710,3.747,0.524
1656,Vietnam,VNM,66000.00,23180.000,2.847,3.747,-0.240


In [30]:
big_mac_2020.to_csv("./data_wrangled/big_mac_price_level_comparison_2020.csv", index=False)